In [58]:
import pandas as pd
import os
import datetime
import time

from evidently.report import Report
from evidently.metrics import ColumnValueRangeMetric
from evidently.collector.config import ReportConfig
from evidently.test_suite import TestSuite
from evidently.tests import TestNumberOfOutRangeValues
from evidently.ui.workspace import Workspace
from evidently.ui.dashboards import DashboardPanelPlot
from evidently.ui.dashboards import PanelValue
from evidently.ui.dashboards import PlotType
from evidently.ui.dashboards import ReportFilter

In [5]:
cur = ref = pd.DataFrame([{"values1": 5.0, "values2": 0.0} for _ in range(10)])

In [6]:
cur

,values1,values2
0,5.0,0.0
1,5.0,0.0
2,5.0,0.0
3,5.0,0.0
4,5.0,0.0
5,5.0,0.0
6,5.0,0.0
7,5.0,0.0
8,5.0,0.0
9,5.0,0.0


In [12]:
report = Report(metrics=[ColumnValueRangeMetric("values1", left=5)], tags=["quality"])
report.run(reference_data=ref, current_data=cur)
rep_config = ReportConfig.from_report(report)

In [17]:
test_report = TestSuite(tests=[TestNumberOfOutRangeValues("values1", left=5)], tags=["quality"])
test_report.run(reference_data=ref, current_data=cur)
test_rep_config = ReportConfig.from_test_suite(test_report)

In [29]:
ws = Workspace("getting started evidently")
project = ws.create_project(name="live monitoring")
project

Project(id=UUID('0193f226-b637-74fd-9135-5edb2bfc5303'), name='live monitoring', description=None, dashboard=DashboardConfig(name='live monitoring', panels=[], tabs=[], tab_id_to_panel_ids={}), team_id=None, org_id=None, date_from=None, date_to=None, created_at=datetime.datetime(2024, 12, 23, 11, 44, 3, 575845))

In [60]:
project.dashboard.add_panel(
        DashboardPanelPlot(
            title="sample_panel",
            filter=ReportFilter(metadata_values={}, tag_values=["quality"]),
            values=[
                PanelValue(metric_id="ColumnValueRangeMetric", field_path="current.share_in_range", legend="current"),
                PanelValue(
                    metric_id="ColumnValueRangeMetric", field_path="reference.share_in_range", legend="reference"
                ),
            ],
            plot_type=PlotType.LINE,
        )
    )
project.save()

Project(id=UUID('0193f226-b637-74fd-9135-5edb2bfc5303'), name='live monitoring', description=None, dashboard=DashboardConfig(name='live monitoring', panels=[DashboardPanelPlot(type='evidently:dashboard_panel:DashboardPanelPlot', id=UUID('0193f23d-163e-73c3-9dd0-14782cc11ff8'), title='sample_panel', filter=ReportFilter(metadata_values={}, tag_values=['quality'], include_test_suites=False), size=<WidgetSize.FULL: 2>, values=[PanelValue(field_path='current.share_in_range', metric_id='ColumnValueRangeMetric', metric_fingerprint=None, metric_args={}, legend='current'), PanelValue(field_path='reference.share_in_range', metric_id='ColumnValueRangeMetric', metric_fingerprint=None, metric_args={}, legend='reference')], plot_type=<PlotType.LINE: 'line'>)], tabs=[], tab_id_to_panel_ids={}), team_id=None, org_id=None, date_from=None, date_to=None, created_at=datetime.datetime(2024, 12, 23, 11, 44, 3, 575845))

In [38]:
from evidently.collector.client import CollectorClient
from evidently.collector.config import CollectorConfig
from evidently.collector.config import IntervalTrigger
from evidently.collector.config import ReportConfig
from evidently.collector.config import RowsCountTrigger

In [48]:
client = CollectorClient("http://localhost:8001")
collector_id = "default"
collector_test_id = "default_test"

In [69]:
#Collector Config

conf = CollectorConfig(
    trigger = RowsCountTrigger(rows_count=100),
    report_config = rep_config,
    project_id = str(project.id)
)
client.create_collector(id=collector_id, collector=conf)


test_conf = CollectorConfig(
    trigger=RowsCountTrigger(rows_count=100), 
    report_config=test_rep_config,
    project_id=str(project.id)
)
client.create_collector(id=collector_test_id, collector=test_conf)

{'id': 'default_test',
 'trigger': {'type': 'evidently:collector_trigger:RowsCountTrigger',
  'rows_count': 100},
 'report_config': {'metrics': [],
  'tests': [{'type': 'evidently:test:TestNumberOfOutRangeValues',
    'is_critical': True,
    'column_name': {'type': 'evidently:base:ColumnName',
     'name': 'values1',
     'display_name': 'values1',
     'dataset': 'main'},
    'left': 5.0}],
  'options': {'color': None, 'render': None, 'data_definition': None},
  'metadata': {},
  'tags': ['quality']},
 'reference_path': None,
 'project_id': '0193f226-b637-74fd-9135-5edb2bfc5303',
 'api_url': 'http://localhost:8000',
 'api_secret': None,
 'cache_reference': True,
 'is_cloud': None,
 'save_datasets': False}

In [72]:
conf = CollectorConfig(
    trigger = IntervalTrigger(interval=5),
    report_config = rep_config,
    project_id = str(project.id)
)
client.create_collector(id=collector_id, collector=conf)

{'id': 'default',
 'trigger': {'type': 'evidently:collector_trigger:IntervalTrigger',
  'interval': 5.0,
  'last_triggered': 0.0},
 'report_config': {'metrics': [{'type': 'evidently:metric:ColumnValueRangeMetric',
    'options': {'color': None, 'render': None, 'data_definition': None},
    'column_name': {'type': 'evidently:base:ColumnName',
     'name': 'values1',
     'display_name': 'values1',
     'dataset': 'main'},
    'left': 5.0,
    'right': None}],
  'tests': [],
  'options': {'color': None, 'render': None, 'data_definition': None},
  'metadata': {},
  'tags': ['quality']},
 'reference_path': None,
 'project_id': '0193f226-b637-74fd-9135-5edb2bfc5303',
 'api_url': 'http://localhost:8000',
 'api_secret': None,
 'cache_reference': True,
 'is_cloud': None,
 'save_datasets': False}

In [73]:
client.set_reference(id=collector_id, reference=ref)
client.set_reference(id=collector_test_id, reference=ref)

{}

In [70]:
def start_sending_data():
    print("Start data loop")
    while True:
        try:
            size = 10
            data = pd.DataFrame([{"values1": 3.0 + datetime.datetime.now().minute % 5, "values2": 0.0} for _ in range(size)])
            client.send_data(collector_id, data)
            client.send_data(collector_test_id, data)
            print("sent")
        except RequestException as e:
            print(f"collector service not available: {e.__class__.__name__}")
        
        time.sleep(1)

In [74]:
start_sending_data()

Start data loop
sent
sent
sent
sent
sent
sent
sent
sent
sent
sent


NameError: name 'RequestException' is not defined